# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [ ]:
# !gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: /content/ElectricCarData.csv
100% 8.20k/8.20k [00:00<00:00, 26.5MB/s]


### Import the required libraries

In [2]:
import pandas as pd
from pygments.lexers.objective import objective

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [29]:
df = pd.read_csv('./ElectricCarData.csv')

In [30]:
df.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


### Encode string variables

In [31]:
df.isnull().sum()

Brand              0
Model              0
AccelSec           0
TopSpeed_KmH       0
Range_Km           0
Efficiency_WhKm    0
FastCharge_KmH     0
RapidCharge        0
PowerTrain         0
PlugType           0
BodyStyle          0
Segment            0
Seats              0
PriceEuro          0
dtype: int64

In [32]:
from sklearn.preprocessing import LabelEncoder


def label_encoder(dataframe: pd.DataFrame, columns: list) -> pd.DataFrame:
    df_copy = dataframe.copy()
    for column in columns:
        le = LabelEncoder()
        mask = df_copy[column].isnull()
        df_copy[column] = le.fit_transform(df_copy[column].astype(str))
        df_copy.loc[mask, column] = None
    return df_copy

In [33]:
categorical = list(df.select_dtypes(include=["object","category"]))
categorical

['Brand',
 'Model',
 'FastCharge_KmH',
 'RapidCharge',
 'PowerTrain',
 'PlugType',
 'BodyStyle',
 'Segment']

In [34]:
df[categorical].sample()

,Brand,Model,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment
85,Fiat,500e Hatchback,330,Yes,FWD,Type 2 CCS,Hatchback,B


In [35]:
df["FastCharge_KmH"].unique()
df["FastCharge_KmH"] = df["FastCharge_KmH"].apply(lambda x: None if x == '-' else int(x))

In [36]:
df["FastCharge_KmH"].isnull().sum()

5

In [38]:
from sklearn.impute import SimpleImputer


def simple_imputer(dataframe: pd.DataFrame, strategy: str, columns) -> pd.DataFrame:
    df_copy = dataframe.copy()
    for column_name in columns:
        imputer = SimpleImputer(strategy=strategy)
        df_copy[column_name] = imputer.fit_transform(df_copy[[column_name]])
    return df_copy

In [41]:
df = simple_imputer(df,"median",["FastCharge_KmH"])

In [42]:
df["FastCharge_KmH"].isnull().sum()

0

In [43]:
categorical = list(df.select_dtypes(include=["object", "category"]))
categorical

['Brand',
 'Model',
 'RapidCharge',
 'PowerTrain',
 'PlugType',
 'BodyStyle',
 'Segment']

In [44]:
df[categorical].sample()

,Brand,Model,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment
15,Volkswagen,ID.3 Pro S,Yes,RWD,Type 2 CCS,Hatchback,C


In [45]:
df = label_encoder(df,categorical)

In [46]:
df[categorical].sample()

,Brand,Model,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment
48,14.0,60.0,1.0,0.0,2.0,2.0,5.0


## Split the dataset for training and testing in ratio 80:20

In [71]:
X = df.drop(["PriceEuro"],axis=1)
Y = df['PriceEuro']

In [72]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [73]:
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor(max_depth=100)

# Train Decision Tree Regressor
regressor.fit(X_train, Y_train)

# #Predict the response for test dataset


DecisionTreeRegressor(max_depth=100)

## Predict the outcomes for X test

In [74]:
y_pred = regressor.predict(X_test)


## Assess the model performance, by using sklearn metrics for regression

In [75]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

l1 = []
l1.append(mean_squared_error(Y_test, y_pred))
l1.append(r2_score(Y_test, y_pred))
l1.append(mean_absolute_error(Y_test,y_pred))


## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [76]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=30, max_depth=5, learning_rate=0.1,objective="reg:squarederror")
model.fit(X_train,Y_train )


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=30, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Predict the outcomes for X test

In [77]:
Y_pred = model.predict(X_test)

## Assess the model performance, by using sklearn metrics for regression

In [78]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

l2 = []
l2.append(mean_squared_error(Y_test, Y_pred))
l2.append(r2_score(Y_test, Y_pred))
l2.append(mean_absolute_error(Y_test, Y_pred))


## Compare the performances of both model for at least three regression metircs

In [79]:
print(l1)
print(l2)

[200699563.33333334, -0.2037760905017758, 8551.904761904761]
[50307695.20368903, 0.6982594728469849, 4995.129185267857]
